# Load files

In [10]:
import pandas as pd
import numpy as np

In [11]:
import pickle

image_survey_metrics = pickle.load( open( "image_survey_metrics.pickle", "rb" ) )


* Load the Y variables

In [53]:
image_survey_metrics.drop_duplicates(inplace=True)

In [54]:
len(image_survey_metrics)

459570

In [55]:
image_survey_metrics.columns

Index([u'image_id', u'image_height', u'image_width', u'image_filter',
       u'image_posted_time', u'data_memorability', u'user_id',
       u'user_followed_by', u'user_follows', u'user_posted_photos',
       u'anp_label', u'anp_sentiment', u'emotion_score', u'emotion_label',
       u'data_amz_label', u'data_amz_label_confidence', u'face_id',
       u'face_gender', u'face_gender_confidence', u'face_age_range_high',
       u'face_age_range_low', u'face_sunglasses', u'face_beard',
       u'face_beard_confidence', u'face_mustache', u'face_mustache_confidence',
       u'face_smile', u'face_smile_confidence', u'eyeglasses',
       u'eyeglasses_confidence', u'face_emo', u'emo_confidence', u'id',
       u'gender', u'born', u'education', u'employed', u'income', u'A_2',
       u'N_1', u'P_1', u'E_1', u'A_1', u'H_1', u'M_1', u'R_1', u'M_2', u'E_2',
       u'LON', u'H_2', u'P_2', u'N_2', u'A_3', u'N_3', u'E_3', u'H_3', u'R_2',
       u'M_3', u'R_3', u'P_3', u'HAP', u'insta_user_id', u'P', u'E', u'

In [ ]:
#different values for same image 
'data_amz_label',

In [79]:
image_survey_metrics[['image_id',
    'image_height',
    'image_width',
    'image_filter',
    'image_posted_time',
    'data_memorability',
    'user_id',
    'user_followed_by',
    'user_follows',
    'user_posted_photos',
    'anp_label',
    'anp_sentiment',
    'emotion_score',
    'emotion_label',
    'data_amz_label',
    'data_amz_label_confidence']].drop_duplicates()

,image_id,image_height,image_width,image_filter,image_posted_time,data_memorability,user_id,user_followed_by,user_follows,user_posted_photos,anp_label,anp_sentiment,emotion_score,data_amz_label,data_amz_label_confidence
0,1372870097060159201_53918317,640.0,640.0,Normal,2016-10-31 01:54:50,0.746274,53918317.0,100.0,174.0,105.0,haunted_house,0.036,0.1634,Human,99.085220
3,1372870097060159201_53918317,640.0,640.0,Normal,2016-10-31 01:54:50,0.746274,53918317.0,100.0,174.0,105.0,haunted_house,0.036,0.1634,People,99.108864
6,1372870097060159201_53918317,640.0,640.0,Normal,2016-10-31 01:54:50,0.746274,53918317.0,100.0,174.0,105.0,haunted_house,0.036,0.1634,Person,99.108871
9,1372870097060159201_53918317,640.0,640.0,Normal,2016-10-31 01:54:50,0.746274,53918317.0,100.0,174.0,105.0,beautiful_hair,0.488,0.1981,Human,99.085220
12,1372870097060159201_53918317,640.0,640.0,Normal,2016-10-31 01:54:50,0.746274,53918317.0,100.0,174.0,105.0,beautiful_hair,0.488,0.1981,People,99.108864
15,1372870097060159201_53918317,640.0,640.0,Normal,2016-10-31 01:54:50,0.746274,53918317.0,100.0,174.0,105.0,beautiful_hair,0.488,0.1981,Person,99.108871
18,1372870097060159201_53918317,640.0,640.0,Normal,2016-10-31 01:54:50,0.746274,53918317.0,100.0,174.0,105.0,fake_fur,-0.916,0.0841,Human,99.085220
21,1372870097060159201_53918317,640.0,640.0,Normal,2016-10-31 01:54:50,0.746274,53918317.0,100.0,174.0,105.0,fake_fur,-0.916,0.0841,People,99.108864
24,1372870097060159201_53918317,640.0,640.0,Normal,2016-10-31 01:54:50,0.746274,53918317.0,100.0,174.0,105.0,fake_fur,-0.916,0.0841,Person,99.108871
27,1372870097060159201_53918317,640.0,640.0,Normal,2016-10-31 01:54:50,0.746274,53918317.0,100.0,174.0,105.0,wicked_witch,0.156,0.2253,Human,99.085220


In [89]:
# 'face_emo','emo_confidence'
test1 = image_survey_metrics.drop(['face_emo','emo_confidence'],axis=1)

In [90]:
test1[test1.columns[test1.iloc[0] != test1.iloc[1]]]

""
0
1
2
3
4
5
6
7
8
9


In [40]:
image_survey_metrics.head()

,image_id,image_height,image_width,image_filter,image_posted_time,data_memorability,user_id,user_followed_by,user_follows,user_posted_photos,...,E,R,M,A,PERMA,N_EMO,P_EMO,imagecount,comment_count,like_count
0,1372870097060159201_53918317,640.0,640.0,Normal,2016-10-31 01:54:50,0.746274,53918317.0,100.0,174.0,105.0,...,7.0,8.0,5.0,5.0,1,8.0,1,105.0,0.0,3.0
1,1372870097060159201_53918317,640.0,640.0,Normal,2016-10-31 01:54:50,0.746274,53918317.0,100.0,174.0,105.0,...,7.0,8.0,5.0,5.0,1,8.0,1,105.0,0.0,3.0
2,1372870097060159201_53918317,640.0,640.0,Normal,2016-10-31 01:54:50,0.746274,53918317.0,100.0,174.0,105.0,...,7.0,8.0,5.0,5.0,1,8.0,1,105.0,0.0,3.0
3,1372870097060159201_53918317,640.0,640.0,Normal,2016-10-31 01:54:50,0.746274,53918317.0,100.0,174.0,105.0,...,7.0,8.0,5.0,5.0,1,8.0,1,105.0,0.0,3.0
4,1372870097060159201_53918317,640.0,640.0,Normal,2016-10-31 01:54:50,0.746274,53918317.0,100.0,174.0,105.0,...,7.0,8.0,5.0,5.0,1,8.0,1,105.0,0.0,3.0


In [41]:
#Convenient way to select multiple columns to get
for c in image_survey_metrics.columns:
    print "'"+c+"',"

'image_id',
'image_height',
'image_width',
'image_filter',
'image_posted_time',
'data_memorability',
'user_id',
'user_followed_by',
'user_follows',
'user_posted_photos',
'anp_label',
'anp_sentiment',
'emotion_score',
'emotion_label',
'data_amz_label',
'data_amz_label_confidence',
'face_id',
'face_gender',
'face_gender_confidence',
'face_age_range_high',
'face_age_range_low',
'face_sunglasses',
'face_beard',
'face_beard_confidence',
'face_mustache',
'face_mustache_confidence',
'face_smile',
'face_smile_confidence',
'eyeglasses',
'eyeglasses_confidence',
'face_emo',
'emo_confidence',
'id',
'gender',
'born',
'education',
'employed',
'income',
'A_2',
'N_1',
'P_1',
'E_1',
'A_1',
'H_1',
'M_1',
'R_1',
'M_2',
'E_2',
'LON',
'H_2',
'P_2',
'N_2',
'A_3',
'N_3',
'E_3',
'H_3',
'R_2',
'M_3',
'R_3',
'P_3',
'HAP',
'insta_user_id',
'P',
'E',
'R',
'M',
'A',
'PERMA',
'N_EMO',
'P_EMO',
'imagecount',
'comment_count',
'like_count',


In [42]:
PERMA_df = image_survey_metrics[['image_id','P',
    'E',
    'R',
    'M',
    'A',
    'PERMA']]

In [43]:
len(PERMA_df)

459570

In [44]:
PERMA_df.head()

,image_id,P,E,R,M,A,PERMA
0,1372870097060159201_53918317,1,7.0,8.0,5.0,5.0,1
1,1372870097060159201_53918317,1,7.0,8.0,5.0,5.0,1
2,1372870097060159201_53918317,1,7.0,8.0,5.0,5.0,1
3,1372870097060159201_53918317,1,7.0,8.0,5.0,5.0,1
4,1372870097060159201_53918317,1,7.0,8.0,5.0,5.0,1


* Load ANP dataset

In [45]:
## use this to load ANP for all images again
# import feather
# anp_df = feather.api.read_dataframe('data_science_case/anp.feather')

In [46]:
#load ANPs from main pickle df
anp_df = image_survey_metrics[['image_id', 'anp_label', 'anp_sentiment', 'emotion_score',
       'emotion_label']]

In [47]:
anp_df.head()

,image_id,anp_label,anp_sentiment,emotion_score,emotion_label
0,1372870097060159201_53918317,haunted_house,0.036,0.1634,fear
1,1372870097060159201_53918317,haunted_house,0.036,0.1634,fear
2,1372870097060159201_53918317,haunted_house,0.036,0.1634,fear
3,1372870097060159201_53918317,haunted_house,0.036,0.1634,fear
4,1372870097060159201_53918317,haunted_house,0.036,0.1634,fear


In [50]:
len(anp_df)

459570

In [51]:
len(anp_df.drop_duplicates())

13275

### Create features based on ANPs for each image

In [30]:
#generate a new df containing the classes corresponding to Butchniks emotions
classes = anp_df.emotion_label.unique()
from sklearn.preprocessing import label_binarize
anp2 = pd.DataFrame(label_binarize(anp_df.emotion_label, classes=classes),columns=classes)
#put the value of emotion score in the corresponding row of the image
anp2 = anp2.multiply(anp_df.emotion_score,axis=0)

In [31]:
anp2.head()

,fear,amazement,sadness,joy,surprise,interest,terror,anger,trust,ecstasy,serenity,pensiveness,grief,acceptance,rage,disgust,boredom,annoyance,distraction,anticipation
0,0.1634,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.1634,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.1634,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.1634,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.1634,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
#concatenate anp2 with image id and ANP sentiment
anp_final = pd.concat([anp_df[['image_id','anp_sentiment']], anp2], axis=1)
anp_final.head()

,image_id,anp_sentiment,amazement,joy,interest,sadness,anger,terror,serenity,fear,...,annoyance,disgust,pensiveness,acceptance,distraction,anticipation,vigilance,loathing,apprehension,admiration
0,951727030670259635_143763900,0.017,0.1760,0.0000,0.0000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,951727030670259635_143763900,0.019,0.0000,0.2113,0.0000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,951727030670259635_143763900,-0.263,0.0000,0.0929,0.0000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,951727030670259635_143763900,-0.031,0.0000,0.0000,0.1304,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,951728575726873168_289794729,0.025,0.1394,0.0000,0.0000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
# # extract the matrix
# import pickle
# with open ('ANP_features.pickle','wb') as f:
#     pickle.dump(image_survey_metrics,f)

In [34]:
anp_final.merge(PERMA_df,on='image_id')

,image_id,anp_sentiment,amazement,joy,interest,sadness,anger,terror,serenity,fear,...,vigilance,loathing,apprehension,admiration,P,E,R,M,A,PERMA
0,1411898784509872288_2264409557,-0.057,0.0000,0.0,0.09,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,6,8.0,7.0,7.0,7.0,6
1,1411898784509872288_2264409557,-0.057,0.0000,0.0,0.09,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,6,8.0,7.0,7.0,7.0,6
2,1411898784509872288_2264409557,-0.057,0.0000,0.0,0.09,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,6,8.0,7.0,7.0,7.0,6
3,1411898784509872288_2264409557,-0.057,0.0000,0.0,0.09,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,6,8.0,7.0,7.0,7.0,6
4,1411898784509872288_2264409557,-0.057,0.0000,0.0,0.09,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,6,8.0,7.0,7.0,7.0,6
5,1411898784509872288_2264409557,-0.057,0.0000,0.0,0.09,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,6,8.0,7.0,7.0,7.0,6
6,1411898784509872288_2264409557,-0.057,0.0000,0.0,0.09,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,6,8.0,7.0,7.0,7.0,6
7,1411898784509872288_2264409557,-0.057,0.0000,0.0,0.09,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,6,8.0,7.0,7.0,7.0,6
8,1411898784509872288_2264409557,-0.057,0.0000,0.0,0.09,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,6,8.0,7.0,7.0,7.0,6
9,1411898784509872288_2264409557,-0.057,0.0000,0.0,0.09,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,6,8.0,7.0,7.0,7.0,6


In [25]:
import seaborn as sns

In [41]:
corr_matrix = anp_final.merge(PERMA_df,on='image_id').corr()

In [43]:
corr_matrix.head()

,anp_sentiment,amazement,joy,interest,sadness,anger,terror,serenity,fear,trust,...,vigilance,loathing,apprehension,admiration,P,E,R,M,A,PERMA
anp_sentiment,1.000000,0.305859,0.174617,-0.049197,-0.226327,-0.130498,-0.137622,-0.012967,-0.098245,-0.008034,...,NaN,NaN,NaN,NaN,0.027321,0.018607,0.049511,0.053158,-0.002704,0.027321
amazement,0.305859,1.000000,-0.353696,-0.223078,-0.272660,-0.155465,-0.090204,-0.097621,-0.079838,-0.057549,...,NaN,NaN,NaN,NaN,-0.005282,-0.031121,-0.002234,0.016314,0.022030,-0.005282
joy,0.174617,-0.353696,1.000000,-0.148710,-0.181762,-0.103637,-0.060132,-0.065077,-0.053222,-0.038363,...,NaN,NaN,NaN,NaN,-0.027162,0.011913,0.051834,0.010826,-0.019340,-0.027162
interest,-0.049197,-0.223078,-0.148710,1.000000,-0.114639,-0.065365,-0.037926,-0.041044,-0.033568,-0.024196,...,NaN,NaN,NaN,NaN,0.009784,-0.009518,-0.026319,-0.043259,-0.007137,0.009784
sadness,-0.226327,-0.272660,-0.181762,-0.114639,1.000000,-0.079893,-0.046355,-0.050167,-0.041028,-0.029574,...,NaN,NaN,NaN,NaN,-0.048177,-0.003223,-0.069353,-0.026430,-0.072407,-0.048177
